# AirLine DATA

In [ ]:
airlinesPath = 'file:///home/icpl12900/Desktop/assignments/Spark_data/airline.csv'
flightPath = 'file:///home/icpl12900/Desktop/assignments/Spark_data/flights.csv'
airportPath = 'file:///home/icpl12900/Desktop/assignments/Spark_data/airports.csv'


In [ ]:
airline = spark.read.format("csv")\
                .option("header","true")\
                .load(airlinesPath)
        

In [ ]:
airport = spark.read.format("csv")\
                .option("header","true")\
                .load(airportPath)
        

In [ ]:
flight = spark.read.format("csv")\
                .option("header","true")\
                .load(flightPath)

In [ ]:
flight

In [ ]:
flight.show(1)

In [ ]:
airline.createOrReplaceTempView('airline')

In [ ]:
airport.createOrReplaceTempView('airport')

In [ ]:
flight.createOrReplaceTempView('flight')

In [ ]:
airline.columns

In [ ]:
airport.columns

In [ ]:
flight.columns

In [ ]:
sqlContext.sql("select airlines from flight").show(1)

### Table is dataFrame so we can use SQL Query as well as DataFrame operations on it

### We want to calculate Total distance for All Flights

In [ ]:
total_distance = sqlContext.sql("select distance from flight")\
                            .agg({"distance":"sum"})\
                            .withColumnRenamed("sum(distance)","total_distance")

In [ ]:
total_distance.show()

### If we want to calculate Total Distance for Particular Airline 

In [ ]:
total_distance_per_airline = sqlContext.sql("select airlines,distance from flight")\
                            .groupBy("airlines")\
                            .agg({"distance":"sum"})\
                            .withColumnRenamed("sum(distance)","total_distance")

In [ ]:
total_distance_per_airline.show(1)

### Departure_delay

In [ ]:
flight.columns

In [ ]:
all_delay_2014 = sqlContext.sql('select date,airlines,departure_delay from flight where departure_delay >0 and year(date) = 2014')

In [ ]:
all_delay_2014.show(1)

In [ ]:
all_delay_2014

In [ ]:
all_delay_2014.createOrReplaceTempView('all_delay_2014_table')

In [ ]:
total_delay_airlines = sqlContext.sql('select count(airlines) from all_delay_2014_table')

In [ ]:
total_no_delay = total_delay_airlines.collect()[0][0]

In [ ]:
total_no_delay

In [ ]:
total_airlines = sqlContext.sql('select count(airlines) from flight')

In [ ]:
total_airlines

In [ ]:
total_airlines.show()

In [ ]:
total_flight = total_airlines.collect()[0][0]

In [ ]:
total_flight

#### Percentage of total delay flights

In [ ]:
percentage = float(total_no_delay) / float(total_flight) * 100

In [ ]:
percentage

##  Find Name of  Airline with Departure Delay 

In [ ]:
flight.columns

In [ ]:
delay_airline = sqlContext.sql('select airlines,departure_delay from flight')\
                        .groupBy('airlines')\
                        .agg({'departure_delay':'avg'})\
                        .withColumnRenamed('avg(departure_delay)','departure_delay')

In [ ]:
delay_airline

In [ ]:
delay_airline.show(1)

In [ ]:
delay_airline_order = delay_airline.orderBy(delay_airline.departure_delay.desc())

In [ ]:
delay_airline_order.show()

## WINDOW FUNCTIONS

#### +----+----+----+----------+----------+

#### |col1|col2|rank|dense_rank|row_number|

#### +----+----+----+----------+----------+

#### |   a|  10|   1|         1|         1|

#### |   a|  10|   1|         1|         2|

#### |   a|  20|   3|         2|         3|

#### +----+----+----+----------+----------+


In [ ]:
product = spark.read.format("csv")\
                .option("header","true")\
                .load('file:///home/icpl12900/Desktop/assignments/Spark_data/products.csv')

In [ ]:
product

In [ ]:
import sys
from pyspark.sql.window import Window
import pyspark.sql.functions as func

In [ ]:
window_product = Window.partitionBy(product['category'])\
                        .orderBy(product['price'].desc())

In [ ]:
window_product

In [ ]:
price_rank = (func.rank().over(window_product))

In [ ]:
price_rank

## RANK()

In [ ]:
product_rank = product.select(product['product'],product['category'],product['price'])\
                    .withColumn('rank',func.rank().over(window_product))

In [ ]:
product_rank

In [ ]:
product_rank.show()

## ROWBETWEEN(-1,0)
##### Here we create window of current row and one before current row

In [ ]:
window_product_row = Window.partitionBy(product['category'])\
                        .orderBy(product['price'].desc())\
                        .rowsBetween(-1,0)

In [ ]:
window_product_row

In [ ]:
price_max = (func.max(product['price']).over(window_product_row))

In [ ]:
price_max

In [ ]:
product.select(product['product'],product['category'],product['price'],price_max.alias("price_max")).show()
        

## rowsBetween(-1,1)
##### Here we select before , current, after

In [ ]:
window_product_row = Window.partitionBy(product['category'])\
                        .orderBy(product['price'].desc())\
                        .rowsBetween(-sys.maxsize,sys.maxsize)

In [ ]:
price_max = (func.max(product['price']).over(window_product_row))

In [ ]:
product.select(product['product'],product['category'],product['price'],price_max.alias("price_max")).show()


## rangeBetween(-sys.maxsize,sys.maxsize)
##### It select all rows within that partition

In [ ]:
window_product_row = Window.partitionBy(product['category'])\
                        .orderBy(product['price'].desc())\
                        .rangeBetween(-sys.maxsize,sys.maxsize)

In [ ]:
price_max = (func.max(product['price']).over(window_product_row))

In [ ]:
product.select(product['product'],product['category'],product['price'],price_max.alias("price_max")).show()


### PRODUCT PRICE DIFFERENCE

In [ ]:
price_difference = (func.max(product['price']).over(window_product_row)  - product['price'])

In [ ]:
product.select(product['product'],product['category'],product['price'],price_difference.alias("price_difference")).show()
